In [6]:
# 한국어 영어 번역 데이터셋
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata

# 실제프로젝트  AI hub 기타등등..
korean_sentences = [
    "안녕하세요",
    "오늘 날씨가 좋아요",
    "저는 학생입니다",
    "이것은 사과입니다",
    "고양이가 자고 있어요",
    "내일 비가 올까요",
    "저는 커피를 좋아해요",
    "그는 의사입니다",
    "이 책은 재미있어요",
    "우리는 친구예요"
]

english_sentences = [
    "hello",
    "the weather is nice today",
    "i am a student",
    "this is an apple",
    "the cat is sleeping",
    "will it rain tomorrow",
    "i like coffee",
    "he is a doctor",
    "this book is interesting",
    "we are friends"
]

In [7]:
# 텍스트 전처리
# unicode 정규화, 특수문자 처리
# NFD Normalization From Decomposition -> 분해가능한 모든 문자를 분해한다.
import unicodedata
unicodedata.normalize('NFD',"e")

'e'

$\acute{e}$

단일문자 U+00E9

$e$ + $\acute{}$

$e$ + 악센트

유니코드 정규화란,

같은 문장처럼 보이지만 내부적으로 다른 바이트 조합을 갖는 경우 

    -> 동일한 내부 표현으로 바꿔주는 과정


In [8]:
def preprocess_stentence(sentence, is_korean = False):
    '''
    Unicode정규화, 특수문자처리
    Args : 
        sentence: 원본
        is_korean : 한국어 여부
    return:
        전처리된 문장
    '''
    sentence = unicodedata.normalize('NFD',sentence)
    if not is_korean:
        sentence = sentence.lower()
    sentence = sentence.strip()
    # 정규식으로 특수문자 전후에 공백 추가
    # 안녕하세요! -> 안녕하세요 !
    # r"([?.!,])"   문자중에 ?.!, 나오면 문자를 그룹으로 캡쳐
    # r" \1 " 캡처한 문자( \1 ) 앞뒤로 공백을 하나씩 넣는다
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]', " ", sentence)
    # 시작 종료토큰 추가
    sentence = '<start>' + sentence + '<end>'
    return sentence

In [9]:
korean_processed = [ preprocess_stentence(kor) for kor in korean_sentences]
english_processed = [preprocess_stentence(eng) for eng in english_sentences]

In [14]:
# 문장을 정수 시퀀스로 변환
# 3 토크나이져
def create_tokenizer(sentence):
    '''
    단어를 정수 인덱스로 변환
    Vocabulary 구축, word to index mapping
    Args:
        sentence : 문장 리스트
    returns : 
        Tokenizer : keras Tokenizer 객체
    '''
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='', # 필터 비활성화
        oov_token = '<unk>'
    )
    # 모든 문장으로 단어사전을 구축
    tokenizer.fit_on_texts(sentence)
    return tokenizer

In [15]:
# 한국어 영어 각각의 토크나이져 생성
korean_tokenizer = create_tokenizer(korean_processed)
english_tokenizer = create_tokenizer(english_processed)

# 단어 사전 크기 확인
korean_vocab_size = len(korean_tokenizer.word_index)+1
english_vocab_size = len(english_tokenizer.word_index)+1
print(f'한국어 사전 크기 : {korean_vocab_size}')
print(f'영어 사전 크기 : {english_vocab_size}')

한국어 사전 크기 : 25
영어 사전 크기 : 30


In [19]:
# 4. 정수시퀀스 변경
from typing import Sequence


def encode_sentences(tokenizer, sentences, maxlen):
    '''
    문장을 고정길이의 정수 시퀀스로 변환
    padding : 짧은 문장을 동일 길이로 맞추는
    Args : 
        tokenizer
        sentence : 문장리스트
        maxlen : 가장 긴 문장
    Returns : 
        패딩된 정수 시퀀스 배열 tf pad_sequence
    '''
    sequence = tokenizer.texts_to_sequences(sentences)   # [I love you so much] [1,5,6,3,7]

    # 길이 맞추기(패딩추가)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequence,
        maxlen = maxlen,
        padding = 'post'
    )
    return padded

- 딥러닝 RNN/LSTM/GRU 전통적인 시계열 처리 기법
    - pre
    - RNN 앞에서부터 읽어 -> 실제 단어가 뒤쪽에 몰려있다 -> hidden state 유효한 문맥을 포함 -> h_last에 정보가 압축
- transfomer 계열을 post
    - 위치정보 -> 앞뒤순서를 그대로 유지해야 자연스럽다 -> 뒤쪽 패딩은 masking 처리

In [21]:
# 최대 시퀀스의 길이 결정 (가장 긴 문장기준)
max_korean_len = max(len(s.split()) for s in korean_processed)
max_english_len = max(len(s.split()) for s in english_processed)
print(f'시퀀스의 최대길이')
print(f'한국어 : {max_korean_len}')
print(f'영어 : {max_english_len}')

# 인코딩 수행
korean_tensor = encode_sentences(korean_tokenizer, korean_processed ,max_korean_len)
english_tensor = encode_sentences(english_tokenizer, english_processed ,max_english_len)
print(f'인코딩 첫번째 문장')
print(f'한국어 문장 : {korean_processed[2]}')
print(f'한국어 인코딩 문장 : {korean_tensor[2]}')
print(f'영어 문장 : {english_processed[2]}')
print(f'영어 인코딩 문장 : {english_tensor[2]}')

BUFFER_SIZE = len(korean_tensor)
BATCH_SIZE = 2

# tensorflow Dataset 객체

시퀀스의 최대길이
한국어 : 3
영어 : 5
인코딩 첫번째 문장
한국어 문장 : <start>저는 학생입니다<end>
한국어 인코딩 문장 : [2 7 0]
영어 문장 : <start>i am a student<end>
영어 인코딩 문장 : [ 4 11  5 12  0]
